In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from config import gkey
import json

In [ ]:
url = "https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html"

In [ ]:
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [ ]:
states = soup.find_all('div', class_='g-state g-cat-all')

In [ ]:
#All States
State = []
Status = []
BusinessUpdate = []
MaskUpdate = []
URL = []

for state in states:
    try:
        name = state.find('div',class_='g-name').text.split('›')[0]
        info = state.find('span', class_="g-text").text
        business = state.find('span', class_='g-rule g-businesses').text
        masks = state.find('span', class_='g-rule g-masks').text
        links = state.find('span',class_='g-link')
        link = links.find('a', href=True)
        if (name):
            State.append(name)
            Status.append(info)
            BusinessUpdate.append(business)
            MaskUpdate.append(masks)
            URL.append(link.get('href'))
    except AttributeError as e:
        print(e)

In [ ]:
COVID_dict={"State":State,"Status":Status,"Business Update":BusinessUpdate,"Mask Update":MaskUpdate,"URL":URL}
COVID=pd.DataFrame(COVID_dict)
#COVID

In [ ]:
COVID['Latitude'] = ''
COVID['Longitude'] = ''
#COVID

In [ ]:
params = {"key": gkey}
for index, row in COVID.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    state = row["State"]
    
    params["address"] = f"{state}"
    
    states_geo = requests.get(base_url, params=params)
    
    states_geo = states_geo.json()
    
    try:
        COVID.loc[index, "Latitude"] = states_geo["results"][0]["geometry"]["location"]["lat"]
        COVID.loc[index, "Longitude"] = states_geo["results"][0]["geometry"]["location"]["lng"]
    except:
        print("Adjustment Needed")
    
#COVID

In [ ]:
#Change the Username
COVID.to_csv(r"C:\Users\bpappas\Downloads\US-COVID.csv",index=False)

In [ ]:
import csv
import json

def make_json(csvFilePath, jsonFilePath):
     
    data = {}
     
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:
             
            key = rows['State']
            data[key] = rows
 
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
         
# Driver Code
 
# Decide the two file paths according to your 
# computer system
csvFilePath = r'C:\Users\bpappas\Downloads\US-COVID.csv'
jsonFilePath = r'C:\Users\bpappas\Documents\Data Work\One-Off\US-COVID.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)